In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

import keras_tuner as kt

In [ ]:
#!pip install -U keras-tuner

In [ ]:
df = pd.read_csv('/content/Diabetes.csv')
df.head()

,pregnancies,plasma glucose concentration,diastolic blood pressure,triceps skinfold thickness,insulin,body mass index,diabetes pedigree function,age,diabetic
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.corr()['diabetic']

pregnancies                     0.221898
plasma glucose concentration    0.466581
diastolic blood pressure        0.065068
triceps skinfold thickness      0.074752
insulin                         0.130548
body mass index                 0.292695
diabetes pedigree function      0.173844
age                             0.238356
diabetic                        1.000000
Name: diabetic, dtype: float64

In [ ]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [ ]:
scaler = StandardScaler()

In [ ]:
x_scale = scaler.fit_transform(X)

In [ ]:
x_scale.shape

(768, 8)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(x_scale,y,test_size=0.2,random_state=1)

In [ ]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data = (X_test,y_test))

Epoch 1/10
20/20 [==============================] - 3s 52ms/step - loss: 0.6939 - accuracy: 0.5179 - val_loss: 0.6514 - val_accuracy: 0.6688
Epoch 2/10
20/20 [==============================] - 0s 14ms/step - loss: 0.6408 - accuracy: 0.6645 - val_loss: 0.6081 - val_accuracy: 0.7468
Epoch 3/10
20/20 [==============================] - 0s 13ms/step - loss: 0.6015 - accuracy: 0.7215 - val_loss: 0.5755 - val_accuracy: 0.7922
Epoch 4/10
20/20 [==============================] - 0s 24ms/step - loss: 0.5712 - accuracy: 0.7394 - val_loss: 0.5495 - val_accuracy: 0.7987
Epoch 5/10
20/20 [==============================] - 0s 13ms/step - loss: 0.5463 - accuracy: 0.7492 - val_loss: 0.5292 - val_accuracy: 0.7987
Epoch 6/10
20/20 [==============================] - 0s 13ms/step - loss: 0.5258 - accuracy: 0.7573 - val_loss: 0.5129 - val_accuracy: 0.7987
Epoch 7/10
20/20 [==============================] - 0s 12ms/step - loss: 0.5101 - accuracy: 0.7573 - val_loss: 0.4999 - val_accuracy: 0.7987
Epoch 8/10
20

## How to select appropriate optimizer

In [ ]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice("optimizer",values=['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.5064935088157654

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 12s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
tuner_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tuner_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
tuner_model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 16ms/step - loss: 0.5270 - accuracy: 0.7573 - val_loss: 0.5291 - val_accuracy: 0.7532
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5095 - accuracy: 0.7622 - val_loss: 0.5116 - val_accuracy: 0.7662
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4965 - accuracy: 0.7655 - val_loss: 0.4985 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4856 - accuracy: 0.7671 - val_loss: 0.4892 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4779 - accuracy: 0.7818 - val_loss: 0.4822 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4719 - accuracy: 0.7785 - val_loss: 0.4771 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4673 - accuracy: 0.7752 - val_loss: 0.4734 - val_accuracy: 0.7857
Epoch 14

## How to select Number of Units/Neurons in Layer

In [ ]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units',min_value=8,max_value=128,step=8)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [ ]:
tuner_neuron = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='model_tuning',project_name='neuron_tuner')

In [ ]:
tuner_neuron.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [ ]:
tuner_neuron.get_best_hyperparameters()[0].values

{'units': 112}

In [ ]:
model_neuron = tuner_neuron.get_best_models(num_models=1)[0]

In [ ]:
model_neuron.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 112)               1008      
                                                                 
 dense_1 (Dense)             (None, 1)                 113       
                                                                 
Total params: 1121 (4.38 KB)
Trainable params: 1121 (4.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model_neuron.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5303 - accuracy: 0.7459 - val_loss: 0.4966 - val_accuracy: 0.7987
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5041 - accuracy: 0.7638 - val_loss: 0.4765 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4874 - accuracy: 0.7704 - val_loss: 0.4661 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4758 - accuracy: 0.7704 - val_loss: 0.4595 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4685 - accuracy: 0.7801 - val_loss: 0.4566 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4623 - accuracy: 0.7752 - val_loss: 0.4533 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4587 - accuracy: 0.7769 - val_loss: 0.4537 - val_accuracy: 0.8117
Epoch 14

## How To Select Number of Layers

In [ ]:
def build_model(hp):
  model= Sequential()
  model.add(Dense(104,activation='relu',input_dim=8))

  for i in range(hp.Int("num_layers",min_value=1,max_value=10)):
    model.add(Dense(104,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner_layer = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='model_tuning',project_name='num_layers')

Reloading Tuner from model_tuning/num_layers/tuner0.json


In [ ]:
tuner_layer.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner_layer.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [ ]:
model_layer = tuner_layer.get_best_models(num_models=1)[0]

In [ ]:
model_layer.fit(X_train,y_train,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 14ms/step - loss: 0.4621 - accuracy: 0.7785 - val_loss: 0.4850 - val_accuracy: 0.7597
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4415 - accuracy: 0.7932 - val_loss: 0.4743 - val_accuracy: 0.7792
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4389 - accuracy: 0.7899 - val_loss: 0.4886 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4316 - accuracy: 0.7883 - val_loss: 0.5100 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4223 - accuracy: 0.7948 - val_loss: 0.4965 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4084 - accuracy: 0.7997 - val_loss: 0.4675 - val_accuracy: 0.7922
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4036 - accuracy: 0.8143 - val_loss: 0.4683 - val_accuracy: 0.7987
Epoch 14

## All in one Hyperparameter Tuning

In [ ]:
def build_model(hp):
  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter==0:

      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                      activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='model_tunning',project_name='all_in_ones')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 08s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 104,
 'activation0': 'relu',
 'dropout0': 0.6,
 'optimizer': 'nadam',
 'units1': 96,
 'activation1': 'relu',
 'dropout1': 0.2,
 'units2': 96,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 112,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 24,
 'activation4': 'sigmoid',
 'dropout4': 0.2}

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dropout (Dropout)           (None, 104)               0         
                                                                 
 dense_1 (Dense)             (None, 96)                10080     
                                                                 
 dropout_1 (Dropout)         (None, 96)                0         
                                                                 
 dense_2 (Dense)             (None, 96)                9312      
                                                                 
 dropout_2 (Dropout)         (None, 96)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 9

In [ ]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 [==============================] - 2s 13ms/step - loss: 0.5302 - accuracy: 0.7410 - val_loss: 0.5010 - val_accuracy: 0.7792
Epoch 7/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5153 - accuracy: 0.7492 - val_loss: 0.4984 - val_accuracy: 0.7857
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5044 - accuracy: 0.7541 - val_loss: 0.4960 - val_accuracy: 0.7792
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5115 - accuracy: 0.7459 - val_loss: 0.4948 - val_accuracy: 0.7857
Epoch 10/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4977 - accuracy: 0.7573 - val_loss: 0.4905 - val_accuracy: 0.7922
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4837 - accuracy: 0.7557 - val_loss: 0.4939 - val_accuracy: 0.7727
Epoch 12/200
20/20 [==============================] - 0s 5ms/step - loss: 0.4949 - accuracy: 0.7508 - val_loss: 0.4921 - val_accuracy: 0.7662
Epoch 13/

In [ ]:
model.save('diabestes_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
